Module imports

In [81]:
# Import modules

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

Data import

In [82]:
# Read data into DataFrames

# Emissions data
nrg_emi_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="CO2 Emissions from Energy", header=2, index_col=0)
# The sheet called "Natural Gas Flaring" is already a part of the calculations for the sheet called "CO2 from Flaring"
flar_emi_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="CO2 from Flaring", header=2, index_col=0)
equi_emi_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="CO2e Methane, Process emissions", header=2, index_col=0)

# Renewable energy data
hydro_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Hydro Generation - TWh", header=2, index_col=0)
solar_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Solar Generation - TWh", header=2, index_col=0)
wind_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Wind Generation - TWh", header=2, index_col=0)
geo_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Geo Biomass Other - TWh", header=2, index_col=0)
biofuel_df = pd.read_excel(io="data/Statistical Review of World Energy Data.xlsx", sheet_name="Biofuels production - PJ", header=2, index_col=0, nrows=47)

Programmatic data processing

In [83]:
def processData(df:pd.DataFrame):
    """
    Get an excel sheet ready for conversion to numpy arrays.

    Parameters:
    - df (pd.DataFrame): a dataframe containing an excel sheet
    """
    #------------------------------ 
    # Remove all irrelevant columns
    #------------------------------

    # Remove all data from before 1990
    # Find the index of the "1990" column
    drop_indx = list(df.columns).index(1990)
    # Get the column labels of all columns left of "1990"
    drop_cols = [df.columns[num] for num in np.arange(0, drop_indx)]
    df = df.drop(columns=drop_cols)

    # Remove data on growth-rate and share
    # Get the column labels of the target columns
    drop_cols = [df.columns[num] for num in [-3, -2, -1]]
    df = df.drop(columns=drop_cols)

    #---------------------------
    # Remove all irrelevant rows
    #---------------------------

    # Remove all rows with any empty cells
    # 0 doesn't make an empty cell
    df = df.dropna()

    # Remove all "Total" and "Other" rows
    # In addition, OECD, Non-OECD, the EU, and the USSR
    # Rationale for removing "Other" rows - some countries in some excel sheets appear
    # individually, but are lumped into an "Other" row in other sheets.
    # There's no possible way for me to know which portions of an
    # "Other" row value belongs to which countries.
    drop_rows = []
    keywords = ["Total", "Other", "OECD", "European Union", "USSR"]
    for row in df.index:
        # Mark a row for dropping if it contains any of the keywords
        if any(keyword in row for keyword in keywords):
            drop_rows.append(row)
    df = df.drop(index=drop_rows)

    # -----------------
    # Convert the units
    # -----------------

    # All CO2 data is currently represented as millions of tonnes
    # Convert all renewable data to kilowatt-hour (kWh)
    # 1 kWh = 3600 kJ
    # 1 PJ = 1000000000000 kJ
    # 1 TWh = 1000000000 kWh

    if (df.index.name) == "Million tonnes of carbon dioxide":
        df = df * 1000000
    elif df.index.name == "Terawatt-hours":
        df = df * 1000000000
    elif df.index.name == "Petajoules":
        df = df * (1000000000000/3600)

    return df

# tonnes = metric ton = 1000 kg


In [84]:
def rowIndices(df:pd.DataFrame):
    """
    Return the row labels of a pd.DataFrame

    Parameters:
    - df (pd.DataFrame): a dataframe containing an excel sheet
    """

    return [row for row in df.index]


In [85]:
# Process dataframes

nrg_emi_df = processData(nrg_emi_df)
flar_emi_df = processData(flar_emi_df)
equi_emi_df = processData(equi_emi_df)

hydro_df = processData(hydro_df)
solar_df = processData(solar_df)
wind_df = processData(wind_df)
geo_df = processData(geo_df)
biofuel_df = processData(biofuel_df)

In [86]:
# Convert to numpy arrays

nrg_emi = nrg_emi_df.to_numpy()
flar_emi = flar_emi_df.to_numpy()
equi_emi = equi_emi_df.to_numpy()
hydro = hydro_df.to_numpy()
solar = solar_df.to_numpy()
wind = wind_df.to_numpy()
geo = geo_df.to_numpy()
biofuel = biofuel_df.to_numpy()

# Get row indices of dataframes
# Only these dataframes were selected because they cover different lists of locations. 
# This leads to different lists of indices.
# len(nrg_emi_indices) = 83
# len(flar_emi_indices) = 49
# len(biofuel_indices) = 24
# The rest of the dataframes share the same index list as nrg_emi_indices
nrg_emi_indices = rowIndices(nrg_emi_df)
flar_emi_indices = rowIndices(flar_emi_df)
biofuel_indices = rowIndices(biofuel_df)

<b>nrg_emi_indices:</b>

['Canada', 'Mexico', 'US', 'Argentina', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Peru', 'Trinidad & Tobago', 'Venezuela', 'Central America', 'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'Azerbaijan', 'Belarus', 'Kazakhstan', 'Russian Federation', 'Turkmenistan', 'Uzbekistan', 'Iran', 'Iraq', 'Israel', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 'Algeria', 'Egypt', 'Morocco', 'South Africa', 'Eastern Africa', 'Middle Africa', 'Western Africa', 'Australia', 'Bangladesh', 'China', 'China Hong Kong SAR', 'India', 'Indonesia', 'Japan', 'Malaysia', 'New Zealand', 'Pakistan', 'Philippines', 'Singapore', 'South Korea', 'Sri Lanka', 'Taiwan', 'Thailand', 'Vietnam']

<b>flar_emi_indices:</b>

['Canada', 'Mexico', 'US', 'Argentina', 'Bolivia', 'Brazil', 'Colombia', 'Peru', 'Trinidad & Tobago', 'Venezuela', 'Denmark', 'Germany', 'Italy', 'Netherlands', 'Norway', 'Poland', 'Romania', 'Ukraine', 'United Kingdom', 'Azerbaijan', 'Kazakhstan', 'Russian Federation', 'Turkmenistan', 'Uzbekistan', 'Bahrain', 'Iran', 'Iraq', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'Syria', 'United Arab Emirates', 'Yemen', 'Algeria', 'Egypt', 'Libya', 'Nigeria', 'Australia', 'Bangladesh', 'Brunei', 'China', 'India', 'Indonesia', 'Malaysia', 'Myanmar', 'Pakistan', 'Thailand', 'Vietnam']

<b>biofuel_indices:</b>

['Canada', 'Mexico', 'US', 'Argentina', 'Brazil', 'Colombia', 'Austria', 'Belgium', 'Finland', 'France', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Spain', 'Sweden', 'United Kingdom', 'Australia', 'China', 'India', 'Indonesia', 'South Korea', 'Thailand']

<b>Shape of nrg_emi_df and nrg_emi:</b>

(83, 33)

<b>Columns of every dataframe:</b>

Index([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022],
      dtype='object')


In [87]:
# Combine it all into one massive 3D numpy array
# 1st dimension - Years. 33 years from 1990-2022 (inclusive)
# 2nd dimension - Countries/Regions. 91 unique countrie/regions
# 3rd dimension - Carbon Neutral features. 8 features (in this order): energy emissions, flaring emissions, CO2 equivalent emissions, 
# hydroelectric production, solar production, wind production, geothermal production, biofuel production
# (33, 91, 8)

# Find every unique country/region
cotry_reg = list(set(nrg_emi_indices + flar_emi_indices + biofuel_indices))
cotry_reg.sort()
print(cotry_reg)

dim_1 = []
for year_indx in range(33):
    dim_2 = []
    for area in cotry_reg:
        if area in nrg_emi_indices:
            indx = nrg_emi_indices.index(area)
            # Extract a float
            a_nrg_emi = nrg_emi[indx][year_indx]
            a_equi_emi = equi_emi[indx][year_indx]
            a_hydro = hydro[indx][year_indx]
            a_solar = solar[indx][year_indx]
            a_wind = wind[indx][year_indx]
            a_geo = geo[indx][year_indx]
        else:
            a_nrg_emi = a_equi_emi = a_hydro = a_solar = a_wind = a_geo = 0.

        if area in flar_emi_indices:
            indx = flar_emi_indices.index(area)
            # Extract a float
            a_flar_emi = flar_emi[indx][year_indx]
        else:
            a_flar_emi = 0.

        if area in biofuel_indices:
            indx = biofuel_indices.index(area)
            # Extract a float
            a_biofuel = biofuel[indx][year_indx]
        else:
            a_biofuel = 0.

        # Is also a set of features
        dim_3 = [a_nrg_emi,
                a_flar_emi,
                a_equi_emi,
                a_hydro,
                a_solar,
                a_wind,
                a_geo,
                a_biofuel]
        dim_2.append(dim_3)
    dim_1.append(dim_2)

print(np.shape(dim_1))
# Statistical Review 
# Full of floats
sr = np.array(dim_1)


['Algeria', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Brazil', 'Brunei', 'Bulgaria', 'Canada', 'Central America', 'Chile', 'China', 'China Hong Kong SAR', 'Colombia', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Eastern Africa', 'Ecuador', 'Egypt', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Japan', 'Kazakhstan', 'Kuwait', 'Latvia', 'Libya', 'Lithuania', 'Luxembourg', 'Malaysia', 'Mexico', 'Middle Africa', 'Morocco', 'Myanmar', 'Netherlands', 'New Zealand', 'Nigeria', 'North Macedonia', 'Norway', 'Oman', 'Pakistan', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russian Federation', 'Saudi Arabia', 'Singapore', 'Slovakia', 'Slovenia', 'South Africa', 'South Korea', 'Spain', 'Sri Lanka', 'Sweden', 'Switzerland', 'Syria', 'Taiwan', 'Thailand', 'Trinidad & Tobago', 'Turkey', 'Turkmenistan', 'US', 'U

In [88]:
def makeLabel(features):
    """
    Make a label for a sample

    - features (np.ndarray): set of 8 features
    """
    # Unit: Tonnes of Carbon Dioxide
    co2 = np.sum(features[:3])
    # Unit: Kilowatt-hours
    renewable = np.sum(features[3:])

    # Electricity reductions emission factor
    # 0.000709 tonnes CO2/kWh
    # Unit: Tonnes of Carbon Dioxide
    renewable *= 0.000709

    # Remaining co2 after being offset by renewable energy production
    rem_co2 = max(co2 - renewable, 0)

    if rem_co2 == co2:
        return 10
    else:
        percent = (rem_co2/co2) * 100
        # Equivalent to np.floor(percent / 10)
        # The label is the tens place of the percentage
        return int(np.floor(percent / 10))


In [89]:
# Make labels for all of the data/samples/examples 
# An individual feature isn't a example, but a location in a particular year is
# Thus, there are 33 * 91 = 3003 examples

# There are 11 possible labels, 0-10
# 0 means carbon neutral is achieved
# 10 means the country is absolutely nowhere near carbon neutrality
labels = np.array([[makeLabel(location) for location in year] for year in sr])
print(labels.shape)

(33, 91)


- 1st Training phase: Train a classifier on the first two labeled years of the data
- 2nd Training phase: Use the classifier and batch active learning on the rest of the unlabeled data until 2021. Examples that would provide the most information will be chosen to get their true label. The remaining examples will get pseudo-labeled
- Evaluate phase: Use the now trained classifier to evaluate the data from 2021 and 2022
- Predict phase: Use time series forecasting (RNN) to predict a country's set of features until 2050. Use the classifier to predict levels of CN

In [90]:
def eclid_dist(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

In [91]:
# Unfinished
def accuracy_fn(pred_labels: np.ndarray, true_labels: np.ndarray):
    try:
        if pred_labels.shape != true_labels.shape:
            raise IndexError
        # Assuming the arguments are 2-dimensional
        

    except IndexError:
        print("There are unequal amount of labels.")

In [92]:
# Split the big dataset into a three subsets: labeled training, unlabeled training, and test

# Labeled training subset
# Years 1990-1991
lab_set = sr[0:2]
lab_set_label = labels[0:2]

# Unlabeled training subset
# Years 1992-2020
unlab_set = sr[2:31]
unlab_set_label = labels[2:31]

# Test subset
# Years 2021-2022
test_set = sr[31::]
test_set_label = labels[31::]

print("Labeled training subset, Years 1990-1991")
print(f"lab_set shape: {lab_set.shape}")
print(f"lab_set_label shape: {lab_set_label.shape}\n")
print("Unlabeled training subset, Years 1992-2020")
print(f"unlab_set shape: {unlab_set.shape}")
print(f"unlab_set_label shape: {unlab_set_label.shape}\n")
print("Test subset, Years 2021-2022")
print(f"test_set shape: {test_set.shape}")
print(f"test_set_label shape: {test_set_label.shape}")

Labeled training subset, Years 1990-1991
lab_set shape: (2, 91, 8)
lab_set_label shape: (2, 91)

Unlabeled training subset, Years 1992-2020
unlab_set shape: (29, 91, 8)
unlab_set_label shape: (29, 91)

Test subset, Years 2021-2022
test_set shape: (2, 91, 8)
test_set_label shape: (2, 91)


In [45]:
for areas, label in zip(lab_set, lab_set_label):
    # print(areas.shape, label.shape)
    print([area.shape for i, area in enumerate(areas)])

[(8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,)]
[(8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8,), (8

In [142]:
# Batch Active Learning
def train_labeled(classifier, lab_data, lab_label):
    """Train on the current labeled dataset

    Parameters:
    - lab_data (ndarray): current labeled data, shape=(# of years, 91 ,8)
    - lab_label (ndarray): labels for current labeled data, shape=(# of years, 91)

    Returns:
    - classifier (classifier type): trained classifier
    """
    for areas, label in zip(lab_data, lab_label):
        # areas.shape = (91, 8)
        # label.shape = (91,)
        classifier.fit(areas, label)

    return classifier

def predict_unlabeled(classifier, batch_data):
    """Predict on the unlabeled data of a year
    
    Parameters:
    - classifier: a trained classifier
    - batch_data (ndarray): a year's worth of unlabeled data, shape=(91, 8)

    Returns:
    - pred_class (ndarray): array of predicted classes, shape=(91,)
    - pred_proba (ndarray): array of array of class probabilities, shape=(91, 11)
    """
    pred_class = np.array(classifier.predict(batch_data))
    pred_proba = np.array(classifier.predict_proba(batch_data))

    return pred_class, pred_proba
    

def batch_active_learning(classifier, lab_data, lab_label, unlab_data, unlab_label, confident_threshold):
    """Train a classifier using batch active learning
    
    Parameters:
    - classifier: a classifier from the scikit-learn (sklearn) module 
    - lab_data (ndarray): the labeled dataset, inital shape=(2, 91, 8)
    - lab_label (ndarray): the labled dataset's labels, inital shape=(2, 91)
    - unlab_data (ndarray): the unlabeled dataset, inital shape=(29, 91, 8)
    - unlab_label (ndarray): the unlabeled dataset's labels, inital shape=(29, 91)
    - confident_threshold (float): threshold for the algorithm to request labels
    """

    index = 0
    while index < 29:
        print(f"Episode {index + 1}")
        uncertain = 0
        classifier = train_labeled(classifier, lab_data, lab_label)

        # Predict on the next batch of unlabeled data
        # A year is a batch
        batch_data = unlab_data[index]
        batch_label = unlab_label[index]
        pred_class, pred_proba = predict_unlabeled(classifier, batch_data)

        # Choose which examples to request a true label for
        # For these examples, replace their predicted label with their true label
        # Remember that the order of examples in pred_class, pred_proba, batch_data, and batch_label are the same
        # Ex. The label information of the example at index 0 of batch_data is found at index 0 of the other arrays
        for idx, probas in enumerate(pred_proba):
            print(probas)
            pred = np.max(probas)
            if pred < confident_threshold:
                uncertain += 1
                pred_class[idx] = batch_label[idx]

        print(f"{uncertain} request made this episode")
        # Reshape batch_data and pred_class for np.append()
        rbatch_data = np.reshape(batch_data, (1, 91, 8))
        rpred_class = np.reshape(pred_class, (1, 91))
        # Add the newly pseudo-labeled, and any true-labeled, examples to the labeled data set
        lab_data = np.append(lab_data, rbatch_data, axis=0)
        lab_label = np.append(lab_label, rpred_class, axis=0)

        index += 1
        
    return classifier

In [123]:
from numpy.random import random
x = random((2, 91, 8))
y = random((91, 8))
y = np.reshape(y, (1, 91, 8))
z = np.append(x, y, axis=0)
z.shape

(3, 91, 8)

In [143]:
# Batch active learning hyperparameters aka model parameters
# These are different from real model parameters that are estimated by the model itself

n_estimators = 20
random_seed = 42
confident_threshold = 0.50

In [141]:
# Gaussian Naive Bayes isn't an option because the data distribution isn't gaussian/normal due to lacking a "symmetric bell shape". 
# Most of the data labels are on the high end of the scale. Thus, the data's bell shape isn't symmetric
# Bernoulli Naive Bayes isn't an option because sample features must be binary-valued (Bernoulli, boolean)
# Multinomial, Complement, and Categorical aren't considered  due to data being classified moreso out of probability rather than certainty.

classifier = RandomForestClassifier(n_estimators=n_estimators, random_state=random_seed)

classifier = batch_active_learning(classifier, 
                                   np.copy(lab_set), np.copy(lab_set_label), 
                                   np.copy(unlab_set), np.copy(unlab_set_label), 
                                   confident_threshold)

Episode 1
[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.95 0.05]
0.95
10.0
[0.   0.   0.   0.   0.   0.   0.05 0.   0.15 0.8  0.  ]
0.8
10.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
1.0
10.0
[0.   0.   0.   0.   0.   0.   0.7  0.   0.05 0.25 0.  ]
0.7
10.0
[0.   0.   0.   0.   0.   0.   0.05 0.   0.   0.85 0.1 ]
0.85
10.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
1.0
10.0
[0.   0.   0.   0.   0.   0.   0.   0.   0.2  0.75 0.05]
0.75
10.0
[0.   0.   0.   0.   0.   0.   0.   0.   0.05 0.9  0.05]
0.9
10.0
[0.   0.   0.05 0.   0.05 0.   0.   0.   0.   0.9  0.  ]
0.9
10.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
1.0
10.0
[0.   0.5  0.   0.05 0.   0.05 0.05 0.   0.25 0.1  0.  ]
0.5
10.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
1.0
10.0
[0.   0.   0.   0.   0.   0.   0.   0.   0.05 0.95 0.  ]
0.95
10.0
[0.  0.  0.  0.  0.  0.3 0.  0.  0.  0.7 0. ]
0.7
10.0
[0.   0.   0.   0.15 0.   0.   0.5  0.25 0.05 0.05 0.  ]
0.5
10.0
[0.   0.   0.   0.05 0.   0.   0.25 0.5  0.05 0.15 0.  ]
0.5
10.0
[0.  0.  0.  0.  0.  0.  0.  

In [33]:
print(set(labels[0]))
print(set(labels[1]))

{0, 1, 2, 3, 5, 6, 7, 8, 9, 10}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


In [53]:
# RNN classifier class
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.linear1 = nn.Linear(in_features=8, out_features=16)
        # nn.ReLU() doesn't need parameters in this case
        self.activation1 = nn.ReLU()
        self.linear2 = nn.Linear(in_features=16, out_features=16)
        self.activation2 = nn.ReLU()
        self.linear3 = nn.Linear(in_features=16, out_features=16)
        self.activation3 = nn.ReLU()
        # self.batchNorm = nn.BatchNorm1d()
        # self.flatten = nn.Flatten()
        # self.dropout1 = nn.Dropout()
        self.dense1 = nn.Linear(in_features=16, out_features=1)
        # self.dropout2 = nn.Dropout()
        # self.dense2 = nn.Linear()
        # self.dropout3 = nn.Dropout()
        # self.dense3 = nn.Linear()
        # self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        # x = self.batchNorm(x)
        # x = self.flatten(x)
        # x = self.dropout1(x)
        x = self.dense1(x)
        # x = self.dropout2(x)
        # x = self.dense2(x)
        # x = self.dropout3(x)
        # x = self.dense3(x)

        return x

In [59]:
device = "cuda" if torch.cuda.is_available() else "cpu"
RNN_classifier = CustomModel().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(RNN_classifier.parameters(), lr=0.0001)

In [ ]:
# Time series forecasting